# Beth Harvey
## Data Analytics Fundamentals Project 7
## February 24, 2023

Objective: Perform multiple linear regression on sklearn's California Housing dataset.

### Load and View the Data

In [ ]:
# Import dataset function
from sklearn.datasets import fetch_california_housing

: 

In [ ]:
# Assign dataset as a variable
california = fetch_california_housing()

: 

In [ ]:
# View dataset desrciption
print(california.DESCR)

: 

Notes: There are no missing values, and all eight features are numerical.

In [ ]:
# View number of samples and features
california.data.shape

: 

In [ ]:
# See number of target values
california.target.shape

: 

The number of samples for the features and the target values are the same.

In [ ]:
# View the feature names of the dataset
california.feature_names

: 

### Data Exploration

In [ ]:
# Import pandas
import pandas as pd

: 

In [ ]:
# Set display options
pd.set_option('display.precision', 4)
pd.set_option('display.max_columns', 9)
pd.set_option('display.width', None)

: 

In [ ]:
# Create DataFrame of the data, target, and feature_names
california_df = pd.DataFrame(california.data, columns = california.feature_names)
california_df['MedHouseValue'] = pd.Series(california.target)

: 

In [ ]:
# View first 5 rows of DataFrame
california_df.head()

: 

In [ ]:
# Get summary statistics for the DataFrame
california_df.describe()

: 

All of the features are fairly normally distributed, with the mean being close to the median value. The biggest difference between the mean and median values is in Population, which appears to be skewed toward the higher end.

In [ ]:
# Create histograms for each column
histograms = california_df.hist(figsize = (14, 14))

: 

Average number of rooms, average number of bedrooms, population, and average occupancy all appear to be heavily affected by some extreme values on the high end. From the descriptive statistics, we see that the highest average number of rooms is 141, the highest average number of bedrooms is 34, the highest population is 35,682, and the highest average occupancy is 1243. These are all significantly higher than the mean and median values for those features.

### Visualizations

In [ ]:
# Select 10% of the samples to make the graphs more readable
sample_df = california_df.sample(frac = 0.1, random_state = 17)

: 

In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import seaborn as sns

: 

In [ ]:
# Set seaborn display options
sns.set(font_scale = 2)
sns.set_style('whitegrid')

: 

In [ ]:
# Display a scatterplot for each feature vs the target value
for feature in california.feature_names:
    plt.figure(figsize = (12, 8))
    sns.scatterplot(data = sample_df, x = feature, y = 'MedHouseValue', hue = 'MedHouseValue', palette = 'cool', 
                    legend = False)

: 

Each graph has a horizontal line of dots at y = 5 because the highest home value option on the survey was "\\$500,000 or more," so any house valued more than \\$500,000 is in the same category.

MedInc appears to have a roughly linear relationship with MedHouseValue. HouseAge doesn't appear to have much of a correlation with MedHouseValue. It is difficult to tell from these graphs due to extreme values, but AveRooms, AveBedrms, Population, and AveOccup may have a slight positive linear relationship with MedHouseValue. Latitude and Longitude both have two clusters. These represent densely-populated areas (Los Angeles and San Francisco areas).

### Train-Test Split

In [ ]:
# Import train-test split function
from sklearn.model_selection import train_test_split

: 

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(california.data, california.target, random_state = 11)

: 

In [ ]:
# Check size of train set
X_train.shape

: 

In [ ]:
# Check size of test set
X_test.shape

: 

### Train Model

In [ ]:
# Import Linear Regression 
from sklearn.linear_model import LinearRegression

: 

In [ ]:
# Create linear regression estimator
linear_regression = LinearRegression()

: 

In [ ]:
# Fit data using train data sets
linear_regression.fit(X = X_train, y = y_train)

: 

In [ ]:
# Get coefficients for each feature
for i, name in enumerate(california.feature_names):
    print(f'{name:>10}: {linear_regression.coef_[i]}')

: 

The average number of bedrooms has the biggest correlation with the median house value. Median income, latitude, and longitude are all pretty similar in magnitude, but income is a positive relationship, while latitude and longitude are both negatively correlated with median house value. The population has almost no effect on median house value, according to this model.

In [ ]:
# Get intercept of model
linear_regression.intercept_

: 

### Test Model

In [ ]:
# Get predicted y values based on X_test
predicted = linear_regression.predict(X_test)

: 

In [ ]:
# Save true y values as "expected"
expected = y_test

: 

In [ ]:
# Look at first 5 predicted values and corresponding expected values
predicted[:5]

: 

In [ ]:
expected[:5]

: 

### Visualizing Expected vs Predicted Values

In [ ]:
# Create DataFrame of expected and predicted values
df = pd.DataFrame()
df['Expected'] = pd.Series(expected)
df['Predicted'] = pd.Series(predicted)

: 

In [ ]:
# Plot expected vs predicted values
figure = plt.figure(figsize = (9, 9))
axes = sns.scatterplot(data = df, x = 'Expected', y = 'Predicted', hue = 'Predicted', palette = 'cool', legend = False)

# Set x and y limits to use same scale for both axes
start = min(expected.min(), predicted.min())
end = max(expected.max(), predicted.max())
axes.set_xlim(start, end)
axes.set_ylim(start, end)

# Add line representing perfect predictions
line = plt.plot([start, end], [start, end], 'k--')

: 

The model appears more likely to predict lower than expected values for higher-priced homes. This could be due to the fact that all houses valued at $500,000 or more are given the same value. 

### Regression Model Metrics

In [ ]:
# Import metrics module
from sklearn import metrics

: 

In [ ]:
# Get R^2 score
metrics.r2_score(expected, predicted)

: 

About 60% of the change in median house value can be explained by the features used in this model.

In [ ]:
# Get mean squared error for model
metrics.mean_squared_error(expected, predicted)

: 

Low MSEs indicate better prediction performance. Since they can range from 0 to infinity, more models would have to be fit for this dataset in order to judge how good this MSE value is.

### Choosing the Best Model

In [ ]:
# Import other estimators for comparison
from sklearn.linear_model import ElasticNet, Lasso, Ridge

: 

In [ ]:
# Create dictionary of estimator objects
estimators = {
    'LinearRegression': linear_regression,
    'ElasticNet': ElasticNet(),
    'Lasso': Lasso(),
    'Ridge': Ridge()
}

: 

In [ ]:
# Import modules for k-fold cross validation
from sklearn.model_selection import KFold, cross_val_score

: 

In [ ]:
# Get mean R^2 values for each estimator using k-fold cross validation
for estimator_name, estimator_object in estimators.items():
    kfold = KFold(n_splits = 10, random_state = 11, shuffle = True)
    scores = cross_val_score(estimator = estimator_object, X = california.data, y = california.target, cv = kfold,
                             scoring = 'r2')
    print(f'{estimator_name:>16}: ' + f'mean of r2 scores = {scores.mean():.3f}')

: 

The LinearRegression and Ridge models have the highest R^2 values for this dataset at 0.599, so either of them would provide the best performance to predict median house values in California. 